Con el código utilizado en clase, implementar una clase de Python similar a esta (https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d ). 
Se le deben pasar como argumentos:

Cantidad de filtros

Tamaño de los filtros

Stride

Tipo de padding (“valid” o “same”)

Debe tener un método “forward”, al cual se le pasa la matriz de entrada y devuelve la salida.

Y un método “backward”, al cual se le pasa una matriz con ∂L/∂O y devuelve dos matrices, una con ∂L/∂F y otra con ∂L/∂X.

(Implementación de capas en Colab: https://colab.research.google.com/drive/1nC98elTULMamrjJt_2VKIWG1NpJB7YqZ)


In [1]:
import numpy as np

In [2]:
# Definimos la función para realizar la convolución dentro de nuestra clase. Se convoluciona la matriz de numpy X con el filtro W
# que tiene el bias "bias"
# También definimos la función que implementa el padding con ceros

def convolve(X, W, bias):
  convol = np.sum(X * W) + float(bias)
  return convol


def zero_padding(X, pad):
    padded = np.pad(X, (pad, pad), mode='constant', constant_values = (0,0))
    return padded



In [91]:
#La matriz de entrada debe ser de dos dimensiones. El filtro debe ser una matriz cuadrada.
class conv2():
    def __init__(self,kernel_nr,kernel_size , Stride, Padding ):
        self.kernel_nr = kernel_nr
        self.kernel_size = kernel_size
        self.Stride = Stride
        self.Padding = Padding
        
    # Definimos el método forward para calcular la salida de los filtros. Utilizamos un solo valor de bias para todos los filtros
    def forward(self, inp, filt, bias):
        # dimensiones de la entrada:
        (n_H, n_W) = inp.shape  
        # dimensiones del filtro (dimensiones: numero de filtros, filter_size, filter zise, cuadrada):
        (n_c, filter_size, filter_size) = filt.shape
        print(n_c)
        if n_c != self.kernel_nr:
            print('No coincide el número de filtros: {} con el declarado en la instancia: {}'. format(n_c, self.kernel_nr))
            return
        # Redefinimos el padding
        if self.Padding == 'VALID':
            padd = 0
        elif self.Padding == 'SAME':
            # Solamente tomamos una dimensión porque en este caso suponemos que la matriz de entrada es cuadrada
            #padd = int((n_H*(self.Stride-1) + self.Stride + filter_size)/2)
            padd = int(((self.Stride-1)*n_H-self.Stride+filter_size)/2)
        else:
            padd = self.Padding
        # dimensiones de salida:
        s_H = int((n_H + 2*padd - filter_size)/self.Stride + 1)
        s_W = int((n_W + 2*padd - filter_size)/self.Stride + 1)
        # Inicialiamos la salida con ceros
        Z = np.zeros([n_c,s_H, s_W])
        print('tamaño de Z:',Z.shape)

        # Realizamos el Padding

        layer_input_padded = zero_padding(inp, padd)
        print('tamaño de la matriz de entrada con padding: ', layer_input_padded.shape)
        
        # Iteramos sobre el eje vertical del volumen de salida
        for h in range(s_H):
            y_start = self.Stride * h
            y_end = y_start + filter_size

            # Iteramos sobre el eje horizontal del volumen de salida
            for w in range(s_W):
                x_start = self.Stride * w
                x_end = x_start + filter_size

                # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
                slice_from_input_padded = layer_input_padded[y_start:y_end, x_start:x_end]
                
                # Itero sobre la cantidad de canales del volumen de salida
                for c in range(n_c):
                    filt_ = filt[c, :, :]     
                    # Computamos la operación de convolución para esta ventana
                    #print(slice_from_input_padded)
                    #print(slice_from_input_padded.shape)
                    Z[c,h,w]= np.float32(convolve(slice_from_input_padded, filt_, bias))        
  
        return Z

    # Definimos el método backward. Las dimensiones de dLdO deben ser las mismas que las de los filtros.
    def backw(self, inp, filt, dLdO):
        (n_H, n_W) = inp.shape  
        # dimensiones del filtro (dimensiones: numero de filtros, filter_size, filter zise, cuadrada):
        (n_c, filter_size, filter_size) = filt.shape

        # Redefinimos el padding
        if self.Padding == 'VALID':
            padd = 0
        elif self.Padding == 'SAME':
            # Solamente tomamos una dimensión porque en este caso suponemos que la matriz de entrada es cuadrada
            #padd = int((n_H*(self.Stride-1) + self.Stride + filter_size)/2)
            padd = int(((self.Stride-1)*n_H-self.Stride+filter_size)/2)
        else:
            padd = self.Padding
        # dimensiones de salida:
        s_H = int((n_H + 2*padd - filter_size)/self.Stride + 1)
        s_W = int((n_W + 2*padd - filter_size)/self.Stride + 1)
        # Inicialiamos la salida con ceros
        dLdF = np.zeros([n_c,s_H, s_W])
        
        layer_input_padded = zero_padding(inp, padd)
        
        for h in range(s_H):
            y_start = self.Stride * h
            y_end = y_start + filter_size
            for w in range(s_W):
                x_start = self.Stride * w
                x_end = x_start + filter_size
                
                # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
                sfip = layer_input_padded[y_start:y_end, x_start:x_end]
                
                for c in range(n_c):
                    dLdO_ = dLdO[c, :, :]     
                    # Computamos la operación de convolución para esta ventana
                    #print(slice_from_input_padded)
                    #print(slice_from_input_padded.shape)
                    dLdF[c,h,w]= np.float32(convolve(sfip, dLdO, 0)) 
        
        # Para calcular dLdX necesitamos rotar 180 grados la matriz del filtro y convolucionar con las matrices dLdO
        dLdX = np.zeros([n_c,s_H, s_W])
        for c in range(n_c):
            filt_ = filt[c, :, :]
            filt_ = np.rot90(filt_, k=2, axes=(0, 1))
            dLdX[c,:,:]= np.float32(convolve(filt_, dLdO, 0)) 
            
        return dLdX, dLdF
        
        
    

In [92]:
prueba4 = np.array([[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]])
prueba3 = np.array([[1,1,1],[1,1,1],[1,1,1]])



In [93]:
m = conv2(kernel_nr=2,kernel_size=2, Stride = 1, Padding = 0)
filt2= np.array([[[2,2,2],[2,2,2],[2,2,2]],[[4,4,4],[4,4,4],[4,4,4]]])
filt3= np.array([[2,2,2],[2,2,2],[2,2,2]])
filt2.shape
dLdO= np.array([[[.3,.2,.2],[.3,.5,.6],[.8,1,2]],[[4,.4,.7],[1,2,3],[0,2,.4]]])

In [94]:
z = m.forward(prueba4, filt2,1)

2
tamaño de Z: (2, 3, 3)
tamaño de la matriz de entrada con padding:  (5, 5)


In [95]:
dLdX, dLdF = m.backw(prueba4, filt2, dLdO)

In [96]:
dLdF

array([[[19.39999962, 19.39999962, 19.39999962],
        [19.39999962, 19.39999962, 19.39999962],
        [19.39999962, 19.39999962, 19.39999962]],

       [[19.39999962, 19.39999962, 19.39999962],
        [19.39999962, 19.39999962, 19.39999962],
        [19.39999962, 19.39999962, 19.39999962]]])

In [97]:
dLdX

array([[[38.79999924, 38.79999924, 38.79999924],
        [38.79999924, 38.79999924, 38.79999924],
        [38.79999924, 38.79999924, 38.79999924]],

       [[77.59999847, 77.59999847, 77.59999847],
        [77.59999847, 77.59999847, 77.59999847],
        [77.59999847, 77.59999847, 77.59999847]]])